In [ ]:
import sys, os, copy, random
from pathlib import Path
import numpy as np
import pandas as pd
import pickle

# Project plumbing — adjust if needed
project_root = "/Users/merlijnbroekers/Desktop/Drone_Interception"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH ok — cwd:", os.getcwd())
print("src/ exists?  ", os.path.exists(Path(project_root) / "src"))

from src.simulation.simulation import Simulation
from src.models.evaders.registry import build_evader
from src.models.pursuers.registry import build_pursuer
from scripts.analysis.utils.metrics_utils import compute_simulation_metrics

In [ ]:
from scripts.analysis.utils.notebook_helpers import AnalysisRunConfig

RUN_CFG = AnalysisRunConfig(
    n_trials=1,
    evader_csv_dir=f"{project_root}/evader_datasets/top_10_csv",
    out_dir=f"{project_root}/figures/dr_sweep",
    results_csv=f"{project_root}/figures/dr_sweep/results.csv",
)

# Toggles for run/load behavior
RUN_TOGGLE = {
    "RECOMPUTE": False,
    "SAVE_RESULTS": False,
}

out_dir = Path(RUN_CFG.out_dir); out_dir.mkdir(parents=True, exist_ok=True)
print(RUN_CFG)

output_dir = Path(RUN_CFG.out_dir)
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# NOTE THIS CONFIG HAS SLIGHTLY DIFFERENT PARAMETERS THAN DEFAULT. THEY CORRESPOND TO THE SYSTEM IDENTIFICATION 
# DONE FOR A DRONE WHICH LATER IN THE PROJECT CRASHED/BROKE. STILL BEBOP2 JUST A DIFFERENT DRONE

CONFIG = {
    "LOG_LEVEL": "INFO",
    "DT": 1 / 100,
    "TOTAL_TIME": 12,
    "TIME_LIMIT": 1200,
    "STOP_ON_INTERCEPTION": False,
    "INTERCEPTION_RADIUS": 0.15,
    "CAPTURE_RADIUS": 0.15,
    "CAPTURE_PENALTY": -2,
    "OUT_OF_BOUNDS_PENALTY": 5,
    "RATE_PENALTY": 0.001,
    "POLICY_KWARGS": dict(net_arch=dict(pi=[64, 64, 64], vf=[64, 64, 64])),
    "OBSERVATIONS": {
        "OBS_MODE": "rel_pos+vel_body",
        "MAX_HISTORY_STEPS": 1,
        "INCLUDE_HISTORY": False,
        "HISTORY_STEPS": 0,
        "INCLUDE_ACTION_HISTORY": False,
        "ACTION_HISTORY_STEPS": 0,
        "ACTION_DIM": 4,
        "OPTIONAL_FEATURES": [
            # "attitude",
            "attitude_mat",
            "rates",
            # "T_force",
            "omega_norm",
        ],
    },
    "reward_type": "effective_gain",
    "SMOOTHING_GAMMA": 0.0,
    "MOTH_FOLDER": "/Users/merlijnbroekers/Desktop/Drone_Interception/evader_datasets/opogona_old/top_moths",
    "PURSUER": {
        "MODEL": "motor",
        "LOG_LEVEL": "DEBUG",
        "INITIAL_POS": [1.0, 0.0, 1.0],
        "INIT_RADIUS": 0.0,
        "INITIAL_VEL": [0.0, 0.0, 0.0],
        "INITIAL_ATTITUDE": [0.0, 0.0, 0.0],
        "INITIAL_RATES": [0.0, 0.0, 0.0],
        "INITIAL_OMEGA": [0.0, 0.0, 0.0],
        "MAX_ACCELERATION": 18,
        "MAX_SPEED": 20,
        "ACTUATOR_TAU": 0.05,
        "POSITION_NOISE_STD": 0.0,
        "VELOCITY_NOISE_STD": 0.00,
        "BUTTER_ACC_FILTER_CUTOFF_HZ": 40,
        "gravity": 9.81,
        "face_evader": False,
        "mass": 1.0,
        "BOUNDARIES": {
            "ENV_BOUNDS": {"x": (-10.0, 10.0), "y": (-10.0, 10.0), "z": (-10.0, 10)},
            "BOUNDARY_MARGIN": 0.3,  # distance from wall where penalty starts
            "BOUNDARY_PENALTY_WEIGHT": 0.25,  # global multiplier (kept)
            "BOUNDARY_MAX_AT_WALL": 1.0,  # per-axis max at the boundary (new)
            "BOUNDARY_MODE": "sum",  # "sum" (default) or "max" (no corner boost)
            "PLANES": None,},
        # [
        #         {
        #             "n": [0.5961325493620492, 0.7253743710122877, 0.3441772878468769],
        #             "p0": [0.0, 0.0, 0.0],
        #         },
        #         {
        #             "n": [0.5961325493620492, -0.7253743710122877, 0.3441772878468769],
        #             "p0": [0.0, 0.0, 0.0],
        #         },
        #         {
        #             "n": [0.8571673007021123, 0.0, -0.5150380749100543],
        #             "p0": [0.0, 0.0, 0.0],
        #         },
        #         {
        #             "n": [-0.017452406437283352, 0.0, 0.9998476951563913],
        #             "p0": [0.0, 0.0, 0.0],
        #         },
        #         {
        #             "n": [0.8660254037844387, 0.0, 0.49999999999999994],
        #             "p0": [0.47631397208144133, 0.0, 0.27499999999999997],
        #         },
        #         {
        #             "n": [-0.8660254037844387, -0.0, -0.49999999999999994],
        #             "p0": [3.464101615137755, 0.0, 1.9999999999999998],
        #         },
        #         {"n": [0.0, 0.0, -1.0], "p0": [0.0, 0.0, 2.0]},
        #         {"n": [0.0, 0.0, 1.0], "p0": [0.0, 0.0, -0.1]},
        #         {"n": [-1.0, 0.0, 0.0], "p0": [2.25, 0.0, 0.0]},
        #         {"n": [1.0, 0.0, 0.0], "p0": [0.55, 0.0, 0.0]},
        #     ],
        # },
        "actuator_time_constants": {
            "p": 2.48e-01,
            "q": 3.18e-01,
            "r": 3.29e-01,
            "T": 9.9e-02,
            "phi": 0.6,
            "theta": 0.6,
        },
        "actuator_limits": {
            "p": (-3.0, 3.0),
            "q": (-3.0, 3.0),
            "r": (-2.0, 2.0),
            "T": (1.41, 20.4),
            "bank_angle": 20.0,
        },
        "delta_a_limits": {
            "min": [-6.0, -6.0, -9.0],
            "max": [6.0, 6.0, 9.0],
        },
        "motor": {
            "k_w": -3.697721e-06,
            "k_x": -8.72e-05,
            "k_y": -6.92e-05,
            "k_p1": 7.80e-05,
            "k_p2": -8.10e-05,
            "k_p3": -7.57e-05,
            "k_p4": 7.43e-05,
            "k_q1": 6.17e-05,
            "k_q2": 6.92e-05,
            "k_q3": -7.24e-05,
            "k_q4": -6.14e-05,
            "k_r1": 7.71e-03,
            "k_r2": -9.59e-03,
            "k_r3": 9.09e-03,
            "k_r4": -7.85e-03,
            "k_r5": 1.87e-03,
            "k_r6": -1.99e-03,
            "k_r7": 1.97e-03,
            "k_r8": -1.75e-03,
            "w_min": 3.089704e02,
            "w_max": 1.174345e03,
            "curve_k": 1.000000e00,
            "tau": 4.791173e-02,
        },
        "drag": {"kx_acc_ctbr": 2.74e-01, "ky_acc_ctbr": 2.13e-01},
        "domain_randomization_pct": {
            k: 0.00
            for k in [
                "g",
                "kx_acc_ctbr",
                "ky_acc_ctbr",
                "taup",
                "tauq",
                "taur",
                "tauT",
                "tauphi",
                "tau_actuator",
                "tautheta",
                "max_accel",
                "max_speed",
                "p_lo",
                "p_hi",
                "q_lo",
                "q_hi",
                "r_lo",
                "r_hi",
                "T_lo",
                "T_hi",
                "bank_angle",
                "delta_a_min",
                "delta_a_max",
                "k_x",
                "k_y",
                "k_w",
                "k_p1",
                "k_p2",
                "k_p3",
                "k_p4",
                "k_q1",
                "k_q2",
                "k_q3",
                "k_q4",
                "k_r1",
                "k_r2",
                "k_r3",
                "k_r4",
                "k_rd1",
                "k_rd2",
                "k_rd3",
                "k_rd4",
                "w_min",
                "w_max",
                "tau",
                "curve_k",
            ]
        }
        | {  # dict merge
            "init_radius": 0.0,
        },
        "CONTROLLER": {
            "type": "rl",  #   "rl" | "frpn" | "lpn" | …
            "policy_path": "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/new_drone_params_abstraction_level_dr/motor/dr10/models/best_model.zip",  #   only for RL
            "params": {  #   whatever the ctor of the class needs
                "lambda_": 180,
                "pp_weight": 0.25,
                "max_acceleration": 18,
            },
            # "params": {
            #     "lambda_": 40,
            #     "k_2": 1,
            #     "v_r": -6,
            #     "max_acceleration": 20,
            # },
        },
    },
    "EVADER": {
        "MODEL": "moth",  # "classic", "moth", "rl", "pliska"
        "EVAL_USE_FILTERED_AS_GT": True,  # False = Meas/GT observation/reward split PRIVALEDGED REWARD, True = Meas/Meas split
        "PLISKA_VEL_FROM_POS": True,
        "PLISKA_SPEED_MULT": 1.0,
        "CSV_FILE": "/Users/merlijnbroekers/Desktop/Drone_Interception/evader_datasets/opogona_old/top_moths/log_itrk32.csv",
        "PLISKA_CSV_FOLDER": "/Users/merlijnbroekers/Desktop/Drone_Interception/evader_datasets/pliska_csv",
        "PLISKA_POSITION_BOUND": 3.0,
        "PATH_TYPE": "figure_eight",
        "RADIUS": 1.0,
        "VELOCITY_MAGNITUDE": 2.0,
        "NOISE_STD": 0.00,
        "NOISE_STD_POS": 0.00,
        "NOISE_STD_VEL": 0.00,
        "FILTER_TYPE": "passthrough",
        "FILTER_PARAMS": {
            "process_noise": 1e-4,
            "measurement_noise": 1e-2,
            "pos_window_samples": 10,
            "vel_window_samples": 5,
            "vel_from_filtered_pos": True,
        },
        "RL_MODEL_PATH": "final_models_selfplay/ALT_0911_1111/round_4_evader/evader_final.zip",
        "MAX_ACCEL": 50,
        "MAX_SPEED": 2.0,
        "INIT_POS": [0, 0, 0],
        "INIT_VEL": [0, 0, 0],
        "BOUNDARIES": {
            "ENV_BOUNDS": {
                "x": (-10.25, 10.25),
                "y": (-10.25, 10.25),
                "z": (-10.25, 10.25),
            },
            "BOUNDARY_MARGIN": 0.5,  # distance from wall where penalty starts
            "BOUNDARY_PENALTY_WEIGHT": 0.5,  # global multiplier (kept)
            "BOUNDARY_MAX_AT_WALL": 1.0,  # per-axis max at the boundary (new)
            "BOUNDARY_MODE": "sum",  # "sum" (default) or "max" (no corner boost)
        },
    },
}


In [ ]:
LEVELS = {
    "motor": {
        "model": "motor",
        "controller": "rl",
        "action_dim": 4,
        "opt_feats": ["attitude_mat", "rates", "omega_norm"],
        "label": "MOTOR",
    },

    "ctbr": {
        "model": "ctbr",
        "controller": "rl",
        "action_dim": 4,
        "opt_feats": ["attitude_mat", "rates", "T_force"],
        "label": "CTBR",
    },
    #     "acc_indi": {
    #     "model": "acc_indi",
    #     "controller": "rl",
    #     "action_dim": 3,
    #     "opt_feats": ["attitude_mat", "rates", "T_force"],
    #     "label": "ACC_INDI",
    # },
    # "PointMass": {
    #     "model": "firstorder",
    #     "controller": "rl",
    #     "action_dim": 3,
    #     "opt_feats": [],
    #     "label": "POINTMASS",
    # },
    }

print("Defined", len(LEVELS), "pursuer setups →", ", ".join(LEVELS))

In [ ]:
POLICIES = {
    "motor": {
        0:  "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/new_drone_params_abstraction_level_dr/motor/dr00/models/best_model.zip",
        10: "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/new_drone_params_abstraction_level_dr/motor/dr10/models/best_model.zip",
        20: "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/new_drone_params_abstraction_level_dr/motor/dr20/models/best_model.zip",
        30: "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/new_drone_params_abstraction_level_dr/motor/dr30/models/best_model.zip",
    },
        "ctbr": {
        0:  "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/abstraction_level_dr/ctbr/dr00/models/best_model.zip",
        10: "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/abstraction_level_dr/ctbr/dr10/models/best_model.zip",
        20: "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/abstraction_level_dr/ctbr/dr20/models/best_model.zip",
        30: "/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/abstraction_level_dr/ctbr/dr30/models/best_model.zip",
    },
    # "PointMass": {
    #     0:  "/Users/merlijnbroekers/Desktop/Drone_Interception/abstraction_level_dr/acc_1order/dr00/models/best_model.zip",
    #     10: "/Users/merlijnbroekers/Desktop/Drone_Interception/abstraction_level_dr/acc_1order/dr10/models/best_model.zip",
    #     20: "/Users/merlijnbroekers/Desktop/Drone_Interception/abstraction_level_dr/acc_1order/dr20/models/best_model.zip",
    #     30: "/Users/merlijnbroekers/Desktop/Drone_Interception/abstraction_level_dr/acc_1order/dr30/models/best_model.zip",
    # },
    #     "acc_indi": {
    #     0:  "/Users/merlijnbroekers/Desktop/Drone_Interception/abstraction_level_dr/acc/dr00/models/best_model.zip",
    #     10: "/Users/merlijnbroekers/Desktop/Drone_Interception/abstraction_level_dr/acc/dr10/models/best_model.zip",
    #     20: "/Users/merlijnbroekers/Desktop/Drone_Interception/abstraction_level_dr/acc/dr20/models/best_model.zip",
    #     30: "/Users/merlijnbroekers/Desktop/Drone_Interception/abstraction_level_dr/acc/dr30/models/best_model.zip",
    # },
    
}

In [ ]:
def build_pursuer_for_level(level_key: str, policy_path: str | None, cfg: dict):
    meta = LEVELS[level_key]

    p_cfg = copy.deepcopy(cfg["PURSUER"])
    p_cfg["MODEL"] = meta["model"]

    if meta["controller"] == "rl":
        if policy_path is None:
            raise ValueError(f"RL level '{level_key}' missing policy path")
        p_cfg["CONTROLLER"] = {"type": "rl", "policy_path": str(policy_path)}
    else:
        # FRPN baseline
        p_cfg["CONTROLLER"] = {"type": "frpn", "params": {"lambda_": 180, "pp_weight": 0.25}}

    return build_pursuer(p_cfg, cfg)

In [ ]:
from scripts.analysis.utils.run_utils import Variant, run_sweep_variants
from scripts.analysis.utils.run_utils import list_evaders 


def run_full_sweep(n_trials: int = RUN_CFG.n_trials):
    evaders = list(list_evaders(RUN_CFG.evader_csv_dir, extra_fields={"NOISE_STD": CONFIG["EVADER"]["NOISE_STD"]}))

    # Expand POLICIES dict into a flat variant list with level and dr_pct
    variants: list[Variant] = []
    for level_key, dr_map in POLICIES.items():
        if level_key not in LEVELS:
            continue
        for dr_pct, policy_path in sorted(dr_map.items(), key=lambda kv: int(kv[0])):
            variants.append(Variant(key=str(level_key), meta={"policy_path": policy_path, "level": level_key, "dr_pct": int(dr_pct)}))

    def build_cfg_fn(ev_cfg, vkey, meta):
        cfg = copy.deepcopy(CONFIG)
        metaL = LEVELS[meta["level"]]
        cfg["OBSERVATIONS"]["ACTION_DIM"] = metaL["action_dim"]
        cfg["OBSERVATIONS"]["OPTIONAL_FEATURES"] = metaL["opt_feats"]
        cfg["EVADER"].update({k: v for k, v in ev_cfg.items() if k != "label"})
        return cfg

    def build_agents_fn(cfg, vkey, meta):
        evader = build_evader(cfg["EVADER"], cfg)
        pursuer = build_pursuer_for_level(meta["level"], meta["policy_path"], cfg)
        return pursuer, evader

    def extra_fields_fn(ev_cfg, vkey, meta, metrics, sim):
        return {"level": meta["level"], "level_label": LEVELS[meta["level"]]["label"], "dr_pct": meta["dr_pct"]}

    df = run_sweep_variants(
        evaders=evaders,
        variants=variants,
        n_trials=n_trials,
        build_cfg_fn=build_cfg_fn,
        build_agents_fn=build_agents_fn,
        compute_metrics_fn=compute_simulation_metrics,
        extra_fields_fn=extra_fields_fn,
        per_evader_seed_bank=True,
    )
    return df


In [ ]:
# trials_path = Path(project_root) / "analysis_out/trials/dr_top_10_evaders.pkl" # domain randomization run for the top 10 Opogona moths
trials_path = Path(project_root) / "analysis_out/trials/dr_top_evader.pkl" # domain randomization run for the top moth, this is also the moth used during the real life testing


In [ ]:
# Compute or load trials based on toggles
if RUN_TOGGLE.get("RECOMPUTE", False):
    trials = run_full_sweep()
    if RUN_TOGGLE.get("SAVE_RESULTS", False):
        with open(trials_path, "wb") as f:
            pickle.dump(trials, f)
else:
    with open(trials_path, "rb") as f:
        trials = pickle.load(f)


In [ ]:
from scripts.analysis.utils.metrics_utils import aggregate_metrics_grouped

aggregate_metrics_grouped(
        trials,
        group_cols=["level_label", "dr_pct"],
        success_only_cols={"first_interception_time", "num_clusters", "total_interceptions"},
    )


In [ ]:
# === Real metrics input for SIM vs REAL comparison ===
# Set this to the CSV produced by collect_real_metrics_from_dirs
REAL_METRICS_CSV = "/Users/merlijnbroekers/Desktop/Drone_Interception/analysis_out/cyber_zoo/real_metrics_domain_randomization.csv"

# # Colors per DR% (edit if you like)
COLOR_BY_DR = {
    0:  "#1f77b4",
    10: "#2ca02c",
    20: "#ff7f0e",
    30: "#d62728",
}

try:
    real_df = pd.read_csv(REAL_METRICS_CSV)
    # normalize dtypes / labels to match sim output
    if "dr_pct" in real_df:
        real_df["dr_pct"] = pd.to_numeric(real_df["dr_pct"], errors="coerce").astype("Int64")
        real_df["dr_pct"] = real_df["dr_pct"].astype(int)
    if "level_label" in real_df:
        real_df["level_label"] = real_df["level_label"].astype(str).str.upper()
except FileNotFoundError:
    print(f"REAL_METRICS_CSV not found at: {REAL_METRICS_CSV} — SIM vs REAL plots will be skipped.")
    real_df = pd.DataFrame()


In [ ]:
from scripts.analysis.utils.plot_utils import plot_sim_real_ctbr_motor_side_by_side

plot_sim_real_ctbr_motor_side_by_side(
    sim_df=trials,
    real_df=real_df,
    ycol="first_interception_time",
    title="First Interception Time Depending on Abstraction Level and Domain Randomization",
    ylabel="Time to first interception (s)",
    savepath=output_dir / "CTBR_MOTOR_SIMvsREAL_FIT_side_by_side.png",
    successes_only=True,
    logy=False,
)

plot_sim_real_ctbr_motor_side_by_side(
    trials, real_df, "time_in_near_miss",
    "Time in Near Miss Depending on Abstraction Level and Domain Randomization", "Percent of time (%)",
    output_dir / "CTBR_MOTOR_SIMvsREAL_near_miss_side_by_side.png",
    successes_only=True, 
    logy=False,
)